In [9]:
import os, sys, shutil, time, os.path as osp, logging, numpy as np
from pathlib import Path
import torch
import matplotlib.pyplot as plt, cycler
import matplotlib
from torch_geometric.data import DataLoader

In [10]:
HGCAL_LDRD_PATH = osp.abspath('../../hgcal_ldrd/src')
PVCNN_PATH = osp.abspath('../../pvcnn')
sys.path.append(HGCAL_LDRD_PATH)

In [11]:
HGCAL_SCRIPTS_PATH = osp.abspath('../../hgcal_ldrd')
sys.path.append(HGCAL_SCRIPTS_PATH)

In [12]:
from datasets.hitgraphs import HitGraphDataset
sys.path.append(PVCNN_PATH)

In [13]:
from scripts import pvcnn_script

In [14]:
script = pvcnn_script.TrainingScript(debug=False)

In [15]:
script.load_checkpoint = '../output/checkpoints/model_checkpoint_PVConvForHGCAL_2562244_9c8b11eb88_alexjschuy.best.pth.tar'
full_dataset, train_dataset, valid_dataset = script.get_full_dataset()
valid_loader = DataLoader(valid_dataset, batch_size=script.valid_batch_size, shuffle=False)
trainer = script.get_trainer()

    INFO:2020-05-19 21:44:34:pvcnn_script:95 Using dataset_path /home/alexjschuy/hgcal_ldrd/scripts/../../data/single-tau
    INFO:2020-05-19 21:44:34:pvcnn_script:110 10000, [ 8000  8000 10000]
    INFO:2020-05-19 21:44:34:pvcnn_script:152 using device cuda
    INFO:2020-05-19 21:44:34:base:31 Model: 
PVConvForHGCAL(
  (point_features): ModuleList(
    (0): PVConv(
      (voxelization): Voxelization(resolution=32, normalized eps = 0)
      (voxel_layers): Sequential(
        (0): Conv3d(5, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
        (1): BatchNorm3d(64, eps=0.0001, momentum=0.1, affine=True, track_running_stats=True)
        (2): LeakyReLU(negative_slope=0.1, inplace=True)
        (3): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
        (4): BatchNorm3d(64, eps=0.0001, momentum=0.1, affine=True, track_running_stats=True)
        (5): LeakyReLU(negative_slope=0.1, inplace=True)
      )
      (point_features): SharedMLP(
       

In [16]:
output_dir = Path('../../data/single-tau/output')
if not output_dir.exists():
    output_dir.mkdir(parents=True)
for i, d in enumerate(valid_dataset):
    one_file_subset = torch.utils.data.Subset(valid_dataset,[i])
    one_file_loader = DataLoader(one_file_subset, batch_size=1, shuffle=False)
    predictions = trainer.predict(one_file_loader)
    input = one_file_subset[0].x.cpu().numpy()
    output = np.zeros((input.shape[0], input.shape[1]+1))
    output[:, :-1] = input
    output[:, -1] = predictions
    output_path = output_dir / f'data_{i}.npy'
    np.save(output_path, output)


  0%|          | 0/10 [00:00<?, ?it/s]

RuntimeError: The expanded size of the tensor (0) must match the existing size (20038) at non-singleton dimension 0.  Target sizes: [0].  Tensor sizes: [20038]